# A brief introduction to the patent system
The patent is a register, tipically a document, that to document a exclusive discovery, invention or method and aims to give to the patent holder exclusive rights over the discovery/invention.

To organize the patents and find a suitable way to structure its information, a commonly used method defines a patent with 2 characteristics:
1. **Task:** the method used in the described patent. In can be compress something or agilize a effect, for example.
2. **Object:** the "target" of the task. It can be a food, a construction material or any other object that, combined with the task, defines the patent.
3. **Kind of Effect:** the kind of effect the Task is causing to the Object.
4. **Effect:** the effect itself being caused to the object.

This method is defined by the Hallbach matrix, that defines a list of Task and Objects that can be extracted from the Title or the Resume of the patent.

# T,O Finder
The T,O Finder is the method that identifies the Task and the Object from a given patent and in this notebook we will construct a method to do such thing using Generative AI.

In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
from tqdm import tqdm
import json
import uuid
import sys

from langchain.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA, ConversationalRetrievalChain, create_qa_with_structure_chain, create_retrieval_chain
from langchain.chains.openai_functions import create_structured_output_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts.base import BasePromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.insert(0, project_root)
os.chdir(project_root)  # Optional: changes working directory to root

from src.genai_utils import TRIZExtractor

load_dotenv()

True

Inserting a ID indexer method to ChromaDB based on discussions in the community. Ref.: https://github.com/langchain-ai/langchain/pull/17938

In [2]:
import langchain_community.vectorstores.chroma as chroma_mod
from typing import (
    Any,
    List,
    Tuple,
)

# Ref.: https://github.com/langchain-ai/langchain/pull/17938
def novo_results_to_docs_and_scores(results: Any) -> List[Tuple[Document, float]]:
    return [
        # TODO: Chroma can do batch querying,
        # we shouldn't hard code to the 1st result
        (
            Document(
                page_content=result[0], metadata=(result[1] | {"id": result[3]}) or {}
            ),
            result[2],
        )
        for result in zip(
            results["documents"][0],
            results["metadatas"][0],
            results["distances"][0],
            results["ids"][0],
        )
    ]

chroma_mod._results_to_docs_and_scores = novo_results_to_docs_and_scores

In [3]:
df_triz = pd.read_csv("data/processed/base_efeitos_físicos_publicada_lemmatized.csv")
df_triz.head()

,TIPO DE EFEITO,TAREFA,OBJETO,EFEITO FÍSICO,SINONIMO 1 EFEITO FISICO,SINONIMO 2 EFEITO FISICO,PT Link,PT Description,Link Wiki (English),TAREFA_lemmatized
0,Aplicação,Apertar,Sólido,Matriz de Halbach,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/Halbach_array,apertar
1,Aplicação,Apertar,Sólido dividido,Matriz de Halbach,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/Halbach_array,apertar
2,Aplicação,Concentrar,Campo,Matriz de Halbach,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/Halbach_array,concentrar
3,Aplicação,Concentrar,Sólido dividido,Matriz de Halbach,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/Halbach_array,concentrar
4,Aplicação,Depositar,Sólido dividido,Matriz de Halbach,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/Halbach_array,depositar


# TRIZ Vector Store Data Preparation

We are preparing text-metadata pairs for a vector store that will be used to match patents with TRIZ (Theory of Inventive Problem Solving) principles. Here's how the data is structured:

**Text Format:** The text is formatted as a natural language sentence following this pattern:

`O "{TAREFA}" é um {TIPO DE EFEITO}, que no {OBJETO} causa {EFEITO FÍSICO}.`

For example:
> "O 'Aquecimento' é um Térmico, que na Água causa Aumento de temperatura."

Each text entry has **metadata** associated containing four key components from TRIZ:
- **tipo_de_efeito**: The type of effect (e.g., Mechanical, Thermal)
- **tarefa**: The task or action being performed
- **objeto**: The object being affected
- **efeito_fisico**: The resulting physical effect
- **id**: an ID used to index and identify a content

This structure allows for:
1. Semantic search through the text descriptions
2. Precise filtering using the metadata fields
3. Mapping between patents and existing TRIZ principles
4. Identification of new derived TRIZ relationships

The entries are created using the `build_entry()` function and then converted into Langchain Document objects for storage in the Chroma vector store.


In [4]:
# Creating the pairs of iniputs
def build_entry(row):
    text = f'O/a "{row["TAREFA"]}" é um/uma {row["TIPO DE EFEITO"]}, que no/na {row["OBJETO"]} causa {row["EFEITO FÍSICO"]}.'
    metadata = {
        "kind_effect": row["TIPO DE EFEITO"],
        "task": row["TAREFA"],
        "object": row["OBJETO"],
        "physical_effect": row["EFEITO FÍSICO"]
    }
    unique_id = str(uuid.uuid4())
    return {"text": text, "metadata": metadata, "id": unique_id}

documents = [build_entry(row) for _, row in df_triz.iterrows()]
documents[:2]

[{'text': 'O/a "Apertar" é um/uma Aplicação, que no/na Sólido causa \xa0Matriz de Halbach.',
  'metadata': {'kind_effect': 'Aplicação',
   'task': 'Apertar',
   'object': 'Sólido',
   'physical_effect': '\xa0Matriz de Halbach'},
  'id': '3e800e0e-fa07-4e35-b9bb-04605486af3f'},
 {'text': 'O/a "Apertar" é um/uma Aplicação, que no/na Sólido dividido causa \xa0Matriz de Halbach.',
  'metadata': {'kind_effect': 'Aplicação',
   'task': 'Apertar',
   'object': 'Sólido dividido',
   'physical_effect': '\xa0Matriz de Halbach'},
  'id': '78f42470-2ca7-4bb4-b939-875d7eac57b0'}]

Now we are going to create a Vectorstore

In [ ]:
docs = [
    Document(page_content=item["text"], metadata=item["metadata"], id=item["id"])
    for item in documents
]

embedding_model = OpenAIEmbeddings()

persist_directory = "notebooks/experiments/chroma_rag_tabular"

# If the vectorstore isnt created
if not os.path.exists(persist_directory):
    vectorstore = Chroma.from_documents(
        documents=docs,
        embedding=embedding_model,
        persist_directory=persist_directory,
        ids=[doc.id for doc in docs],
    )
    vectorstore.persist()

else:
    print("Vectorstore already exists")
    vectorstore = Chroma(
        persist_directory=persist_directory,
        embedding_function=embedding_model,
    )

persist_directory

/tmp/ipykernel_439828/1396729805.py:6: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings()
/tmp/ipykernel_439828/1396729805.py:18: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


'notebooks/experiments/chroma_rag_tabular'

In [ ]:
from langchain.vectorstores import Chroma

# Reopens the vectorstore
vectorstore = Chroma(
    persist_directory="notebooks/experiments/chroma_rag_tabular",
    embedding_function=embedding_model
)

# Similarity search
query = "Qual tarefa pode causar aumento da temperatura?"
results = vectorstore.similarity_search(query, k=2)

for doc in results:
    print(doc)
    print("Metadados:", doc.metadata)
    print("id: ", doc.metadata['id'])


/tmp/ipykernel_439828/3533990342.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


page_content='O/a "Aquecer" é um/uma Efeito, que no/na Sólido causa Dilatação térmica.' metadata={'kind_effect': 'Efeito', 'task': 'Aquecer', 'object': 'Sólido', 'physical_effect': 'Dilatação térmica', 'id': 'b388e817-e358-4577-882f-6798228ee0fd'}
Metadados: {'kind_effect': 'Efeito', 'task': 'Aquecer', 'object': 'Sólido', 'physical_effect': 'Dilatação térmica', 'id': 'b388e817-e358-4577-882f-6798228ee0fd'}
id:  b388e817-e358-4577-882f-6798228ee0fd
page_content='O/a "Corroer" é um/uma Efeito, que no/na Sólido causa Hipertermia.' metadata={'task': 'Corroer', 'physical_effect': 'Hipertermia', 'kind_effect': 'Efeito', 'object': 'Sólido', 'id': 'f2ba23e5-0fbd-4c76-ae55-beac78165d05'}
Metadados: {'task': 'Corroer', 'physical_effect': 'Hipertermia', 'kind_effect': 'Efeito', 'object': 'Sólido', 'id': 'f2ba23e5-0fbd-4c76-ae55-beac78165d05'}
id:  f2ba23e5-0fbd-4c76-ae55-beac78165d05


In [7]:
from pydantic import BaseModel

class TrizSchema(BaseModel):
    derived_from: str | None
    kind_effect: str
    task: str
    object: str
    physical_effect: str

## Explanation of the Prompt Construction and QA Chain

This section of the code builds the prompt and configures the retrieval-augmented generation (RAG) chain for extracting structured TRIZ information from patent texts using a language model.

### Prompt Construction

- The `template` variable defines detailed instructions for the LLM in Portuguese. It explains:
  - The context: the model will receive a patent title and abstract.
  - The goal: connect the patent to a TRIZ relationship using the TRIZ base (object, task, effect type, and physical effect).
  - The model can create new TRIZ relationships if none fit, but must indicate the closest existing element in the `derived_from` field.
  - Rules for when to use or derive elements, and how to fill the output fields.
  - The required output format: a JSON object with four main fields (`kind_effect`, `task`, `object`, `physical_effect`), each with a value and an optional `derived_from`.
  - An example JSON output is provided, with all curly braces escaped to avoid template variable conflicts.

### PromptTemplate and Chain Setup

- The `PromptTemplate` is created with the template and the expected input variables (`context` and `question`).
- The `PydanticOutputParser` ensures the LLM output matches the `TrizSchema` structure.
- The `combine_docs_chain` is created to combine retrieved documents and the prompt for the LLM.
- The `qa_chain` is a retrieval chain that:
  - Uses the vectorstore retriever to fetch relevant TRIZ context.
  - Passes the context and the patent input to the LLM using the prompt.
  - Parses the output into the structured schema.

### Summary

This setup enables the LLM to:
- Receive a patent description.
- Retrieve relevant TRIZ principles from the vectorstore.
- Output a structured JSON mapping the patent to TRIZ elements, including any new derived elements, following strict formatting and validation rules.

In [8]:
df_patents = pd.read_csv("data/processed/patentes_inpi_english_matched.csv")
df_patents.head()

,id_pedido,data_deposito,titulo,ipc,url,resumo,classifica_ipc,titulo_english,match_top_10_title
0,BR 11 2021 018393 0,02/03/2020,TRATAMENTO DE COLISÕES EM UPLINK,H04L 1/18,https://busca.inpi.gov.br/pePI/servlet/Patente...,"A presente invenção se refere a métodos, sis...",H04L 1/18,Treatment of collisions in Uplink,"{'Move', 'Break Down', 'Change Phase', 'Separa..."
1,BR 11 2021 018071 0,02/03/2020,ALOJAMENTO DE VELA DE IGNIÇÃO COM PROTEÇÃO ANT...,H01T 13/14,https://busca.inpi.gov.br/pePI/servlet/Patente...,ALOJAMENTO DE VELA DE IGNIÇÃO COM PROTEÇÃO A...,H01T 13/14 ; H01T 13/20 ; H01T 13/32 ; H0...,"In this case, it is necessary to ensure that y...","{'Move', 'Break Down'}"
2,BR 11 2021 016947 4,02/03/2020,ANTICORPOS QUE RECONHECEM TAU,C07K 16/18,https://busca.inpi.gov.br/pePI/servlet/Patente...,ANTICORPOS QUE RECONHECEM TAU. A invenção fo...,C07K 16/18 ; G01N 33/68,Antibodies that recognize you,"{'Move', 'Break Down', 'Change Phase', 'Cool',..."
3,BR 10 2020 004169 0,02/03/2020,AQUECEDOR DE AR A LENHA COM DUPLA EXAUSTÃO PAR...,F24H 3/00,https://busca.inpi.gov.br/pePI/servlet/Patente...,AQUECEDOR DE AR A LENHA COM DUPLA EXAUSTAO P...,F24H 3/008 ; F24H 4/06,Air heater with double exhaust to be used in a...,"{'Move', 'Break Down', 'Expand', 'Separate', '..."
4,BR 11 2021 006234 3,02/03/2020,BIBLIOTECAS DE CÉLULAS ÚNICAS E NÚCLEOS ÚNICOS...,C12N 15/10,https://busca.inpi.gov.br/pePI/servlet/Patente...,BIBLIOTECAS DE CÉLULAS ÚNICAS E NÚCLEOS ÚNIC...,C12N 15/10,Unique cell libraries and unique high-end nucl...,"{'Remove', 'Break Down', 'Move', 'Concentrate'..."


In [9]:
extractor = TRIZExtractor(vectorstore=vectorstore, verbose=True)

In [10]:
i = 2
title = df_patents.iloc[i]["titulo"]
abstract = df_patents.iloc[i]["resumo"]

query = f"""Título: {title}
Abstract: {abstract}"""

print(query)

Título: ANTICORPOS QUE RECONHECEM TAU
Abstract:   ANTICORPOS QUE RECONHECEM TAU. A invenção fornece anticorpos que se ligam especificamente à tau. Os anticorpos inibem ou retardam patologias associadas à tau e deterioração sintomática associada.


In [11]:
extractor.run(title=title, abstract=abstract)

Query to be processed
 Título: ANTICORPOS QUE RECONHECEM TAU
        Abstract:   ANTICORPOS QUE RECONHECEM TAU. A invenção fornece anticorpos que se ligam especificamente à tau. Os anticorpos inibem ou retardam patologias associadas à tau e deterioração sintomática associada. 
**********


Extraction result
 kind_effect='Inibição de patologias associadas à proteína tau' task='Inibir ou retardar patologias' object='Proteína tau' physical_effect='Ligação específica de anticorpos à proteína tau' 
**********


Similar samples result
 [(Document(metadata={'task': 'Detectar', 'physical_effect': 'Polímero eletroativo', 'kind_effect': 'Aplicação', 'object': 'Campo', 'id': '2da27b88-eb49-4376-a770-8dce76699176'}, page_content='O/a "Detectar" é um/uma Aplicação, que no/na Campo causa Polímero eletroativo.'), 0.38666343688964844), (Document(metadata={'task': 'Proteger', 'kind_effect': 'Aplicação', 'physical_effect': 'Substância aglutinante', 'object': 'Sólido', 'id': '59546625-a236-4b27-aac6-6e44

{'derived_from': None,
 'effect': 'Aplicação',
 'task': 'Detectar',
 'object': 'Campo',
 'physical_effect': 'Polímero eletroativo'}

### Explanation: `insert_derived_elements` Function

The `insert_derived_elements` function is responsible for identifying and storing new (derived) TRIZ elements generated by the LLM that are not present in the original TRIZ base. Here’s how it works:

1. **Purpose**  
   - To check the structured output from the LLM for any fields (`kind_effect`, `task`, `object`, `physical_effect`) that are marked as "derived" (i.e., their `derived_from` attribute is not `None`).
   - To add these new derived elements to the vectorstore for future retrieval and semantic search.

2. **How it Works**  
   - The function iterates over each of the four main TRIZ fields in the structured output.
   - For each field, it checks if the `derived_from` attribute is filled (not `None`). If so, it collects information about the derived element (field name, value, and what it was derived from).
   - If any derived elements are found, it creates a new row (dictionary) with the values for `TAREFA`, `TIPO DE EFEITO`, `OBJETO`, and `EFEITO FÍSICO` from the structured output.
   - It then uses the `build_entry` function to create a new document (with text and metadata) and adds this document to the vectorstore.
   - Finally, it prints information about each derived element added and the text that was inserted into the vectorstore.

3. **Why is this Useful?**  
   - This approach allows the system to dynamically expand the TRIZ knowledge base with new, LLM-suggested relationships, while keeping track of their origin.
   - It ensures that the vectorstore remains up-to-date and can support richer, more flexible semantic search and reasoning in future queries.

**In summary:**  
Whenever the LLM proposes a new TRIZ element (not found in the base), this function logs and stores it, making the knowledge base grow automatically as new patents are processed.

In [ ]:
# Calcula 10% do tamanho total
sample_size = int(len(df_patents) * 0.1)
df_patents_sample = df_patents.sample(n=sample_size, random_state=42)

results = []

for idx, row in tqdm(df_patents_sample.iterrows(), total=len(df_patents_sample)):
    try:
        title = row["titulo"]
        abstract = row["resumo"]

        # Use the extractor object to process and insert derived elements
        result = extractor.run(title=title, abstract=abstract)

        # Save fields to DataFrame
        df_patents_sample.at[idx, "kind_effect"] = result.get("effect", "")
        df_patents_sample.at[idx, "task"] = result.get("task", "")
        df_patents_sample.at[idx, "object"] = result.get("object", "")
        df_patents_sample.at[idx, "physical_effect"] = result.get("physical_effect", "")
        df_patents_sample.at[idx, "derived_from"] = result.get("derived_from", "")

        print("Structured output:", result)

        results.append(result)
    except Exception as e:
        print(f"Erro no índice {idx}: {e}")
        df_patents_sample.at[idx, "kind_effect"] = "Erro"
        df_patents_sample.at[idx, "task"] = "Erro"
        df_patents_sample.at[idx, "object"] = "Erro"
        df_patents_sample.at[idx, "physical_effect"] = "Erro"
        df_patents_sample.at[idx, "derived_from"] = "Erro"
        results.append({"kind_effect": "Erro", "task": "Erro", "object": "Erro", "physical_effect": "Erro", "derived_from": "Erro"})

# Salva os resultados
df_patents_sample.to_csv("data/processed/patents_inpi_llm_matched_10percent.csv", index=False)

  0%|          | 0/329 [00:00<?, ?it/s]

Query to be processed
 Título: CONFIGURAÇÃO APLICADA EM SOBRETAMPA METÁLICA
        Abstract:   RESUMO CONFIGURAÇÃO APLICADA EM SOBRETAMPA METÁLICA O presente modelo de utilidade refere-se a uma configuração aplicada em sobre tampa metálica, compreendendo fixação através do atrito de travas com a recravação e o corpo da embalagem, vedação através da material emborrachado, plastisol ou similares, e rebaixos para empilhamento de produto, sendo aplicada em embalagens com tampas e utilizada após a abertura da tampa primária (que faz o fechamento e garante a hermeticidade da embalagem), permitindo a retampagem e garantindo a vida útil do produto envasado após a abertura da tampa primária. 
**********


Extraction result
 kind_effect='Vedação e proteção do conteúdo da embalagem após abertura' task='Retampar e vedar' object='Embalagem com tampa metálica' physical_effect='Ação mecânica de travamento por atrito e vedação por compressão de material emborrachado' 
**********




  0%|          | 1/329 [00:04<22:03,  4.03s/it]

Similar samples result
 [(Document(metadata={'kind_effect': 'Aplicação', 'task': 'Apertar', 'object': 'Sólido dividido', 'physical_effect': 'Espuma metálica', 'id': '10ee990e-94e2-4d35-8343-a5af16115392'}, page_content='O/a "Apertar" é um/uma Aplicação, que no/na Sólido dividido causa Espuma metálica.'), 0.32272523641586304), (Document(metadata={'kind_effect': 'Aplicação', 'physical_effect': 'Estrutura metalorgânica', 'task': 'Apertar', 'object': 'Líquido', 'id': '5bd9e3b8-648a-4340-882f-ea5a2dd808df'}, page_content='O/a "Apertar" é um/uma Aplicação, que no/na Líquido causa Estrutura metalorgânica.'), 0.3233375549316406), (Document(metadata={'kind_effect': 'Aplicação', 'physical_effect': 'Espuma metálica', 'object': 'Líquido', 'task': 'Apertar', 'id': 'd406a023-3d2d-442b-928f-5c2a6fa8d1db'}, page_content='O/a "Apertar" é um/uma Aplicação, que no/na Líquido causa Espuma metálica.'), 0.32858237624168396)] 
**********


Best_doc: page_content='O/a "Apertar" é um/uma Aplicação, que no/na S

  1%|          | 2/329 [00:07<21:02,  3.86s/it]

...

100%|█████████▉| 328/329 [12:44<00:02,  2.02s/it]

Similar samples result
 [(Document(metadata={'object': 'Símbolos de enlace ascendente em canal físico compartilhado', 'derived_from': 'O/a "Orientar" é um/uma Aplicação, que no/na Campo causa Fibra óptica.', 'physical_effect': 'Multiplexação e alocação dinâmica de recursos de frequência e tempo', 'task': 'Transmitir dados de enlace ascendente', 'kind_effect': 'Otimização do uso de recursos de canal físico compartilhado', 'id': 'f102ecb1-83d9-4995-8d11-c0f948e8e47a'}, page_content='O "Transmitir dados de enlace ascendente" é um Otimização do uso de recursos de canal físico compartilhado, que no Símbolos de enlace ascendente em canal físico compartilhado causa Multiplexação e alocação dinâmica de recursos de frequência e tempo.'), 0.2882675528526306), (Document(metadata={'task': 'Gerar e enviar informação de confirmação', 'object': 'Recurso de comunicação em enlace lateral', 'physical_effect': 'Transmissão de dados sem fio', 'derived_from': 'O/a "Detectar" é um/uma Aplicação, que no/na S

100%|██████████| 329/329 [12:46<00:00,  2.33s/it]

Similar samples result
 [(Document(metadata={'object': 'Sólido dividido', 'physical_effect': 'Compostagem', 'task': 'Concentrar', 'kind_effect': 'Aplicação', 'id': '6998e393-95e4-4c20-a087-72683de8efcb'}, page_content='O/a "Concentrar" é um/uma Aplicação, que no/na Sólido dividido causa Compostagem.'), 0.3803565204143524), (Document(metadata={'physical_effect': 'Reação química de transporte', 'kind_effect': 'Aplicação', 'task': 'Concentrar', 'object': 'Sólido dividido', 'id': '986136cb-45d8-419e-8680-ca4aa5c3a7a7'}, page_content='O/a "Concentrar" é um/uma Aplicação, que no/na Sólido dividido causa Reação química de transporte.'), 0.38093888759613037), (Document(metadata={'physical_effect': 'Reação química de transporte', 'object': 'Sólido', 'task': 'Concentrar', 'kind_effect': 'Aplicação', 'id': '6068396c-4120-4d0d-add7-c56b0c4d22d9'}, page_content='O/a "Concentrar" é um/uma Aplicação, que no/na Sólido causa Reação química de transporte.'), 0.3855287432670593)] 
**********


Best_doc: 

OSError: Cannot save file into a non-existent directory: '../../data/processed'

In [17]:

df_patents_sample.to_csv("data/processed/patents_inpi_llm_matched_10percent.csv", index=False)

In [14]:
# Análise dos campos kind_effect, task, object e physical_effect em df_patents_sample

# Função para contar repetições e mostrar os mais frequentes
def analyze_repetitions(df, column):
    counts = df[column].value_counts()
    repeated = counts[counts >= 2]
    print(f"\nCampo: {column}")
    print(f"Total de valores únicos: {counts.shape[0]}")
    print(f"Valores que se repetem pelo menos 2 vezes: {repeated.shape[0]}")
    print("Top 5 mais frequentes:")
    print(repeated.head(5))

# Campos a analisar
fields = ["kind_effect", "task", "object", "physical_effect"]

for field in fields:
    analyze_repetitions(df_patents_sample, field)


Campo: kind_effect
Total de valores únicos: 29
Valores que se repetem pelo menos 2 vezes: 15
Top 5 mais frequentes:
kind_effect
Aplicação                                                               230
Efeito                                                                   25
Otimização do uso de recursos de canal físico compartilhado              16
Evitar desperdício de recursos e assegurar desempenho de transmissão      8
Supressão ou destruição de células cancerígenas                           7
Name: count, dtype: int64

Campo: task
Total de valores únicos: 56
Valores que se repetem pelo menos 2 vezes: 40
Top 5 mais frequentes:
task
Detectar      38
Produzir      27
Segurar       26
Desmembrar    21
Apertar       16
Name: count, dtype: int64

Campo: object
Total de valores únicos: 32
Valores que se repetem pelo menos 2 vezes: 18
Top 5 mais frequentes:
object
Sólido dividido                                                93
Líquido                                               

In [15]:
docs_data = [
    {
        "text": doc["text"],
        "kind_effect": doc["metadata"]["kind_effect"],
        "task": doc["metadata"]["task"],
        "object": doc["metadata"]["object"],
        "physical_effect": doc["metadata"]["physical_effect"],
        "id": doc["id"]
    }
    for doc in documents
]

# Cria o DataFrame e salva em CSV
df_docs = pd.DataFrame(docs_data)
df_docs.to_csv("data/processed/triz_documents_reference_ai.csv", index=False)

In [16]:
# Verifica se algum elemento em df_patents_sample["derived_from"] não está em docs_data["text"]
# e ao mesmo tempo não possui "derived_from=True"

# Carrega os textos dos documentos de referência
reference_texts = set(df_docs["text"].tolist())

# Filtra os elementos do DataFrame de patentes
missing_texts = []
for idx, row in df_patents_sample.iterrows():
    derived_from = row.get("derived_from", "")
    # Ignora linhas de erro ou derived_from vazio
    if not derived_from or derived_from == "Erro":
        continue
    # Verifica se derived_from está nos textos de referência
    if derived_from not in reference_texts and row.get("derived_from", True) != True:
        missing_texts.append(derived_from)

print("Elementos em 'derived_from' que não estão em docs_data['text'] e não possuem 'derived_from=True':")
for text in missing_texts:
    print(text)

Elementos em 'derived_from' que não estão em docs_data['text'] e não possuem 'derived_from=True':


In [5]:
df_patents_sample=pd.read_csv("data/processed/patents_inpi_llm_matched_10percent.csv")
df_patents_sample.head()

,id_pedido,data_deposito,titulo,ipc,url,resumo,classifica_ipc,titulo_english,match_top_10_title,kind_effect,task,object,physical_effect,derived_from
0,BR 20 2020 001664 0,24/01/2020,CONFIGURAÇÃO APLICADA EM SOBRETAMPA METÁLICA,B65D 43/02,https://busca.inpi.gov.br/pePI/servlet/Patente...,RESUMO CONFIGURAÇÃO APLICADA EM SOBRETAMPA M...,B65D 43/02,Configuration applied in metallic substrate,"{'Move', 'Break Down', 'Remove', 'Change Phase...",Aplicação,Apertar,Sólido dividido,Espuma metálica,NaN
1,BR 11 2021 015521 0,11/02/2020,"MISTURAS DE PESTICIDAS, MÉTODO PARA CONTROLAR ...",A01N 43/58,https://busca.inpi.gov.br/pePI/servlet/Patente...,"MISTURAS DE PESTICIDAS, MÉTODO PARA CONTROLA...",A01N 43/58 ; A01N 33/20 ; A01N 37/34 ; A0...,"Pesticide mixture, method for controlling phyt...","{'Produce', 'Prevents', 'Change Phase', 'Mix',...",Aplicação,Misturar,Sólido,Substância aglutinante,NaN
2,BR 11 2021 012872 7,08/01/2020,"CHAPA DE AÇO ELÉTRICO COM GRÃO ORIENTADO, MÉTO...",C21D 8/12,https://busca.inpi.gov.br/pePI/servlet/Patente...,"CHAPA DE AÇO ELÉTRICO COM GRÃO ORIENTADO, MÉ...",C21D 8/12 ; C22C 38/00 ; C22C 38/60 ; H01...,"Electric power sheet with large-oriented, meth...","{'Produce', 'Move', 'Break Down', 'Change Phas...",Aplicação,Orientar,Sólido dividido,Pó ferromagnético,NaN
3,BR 11 2021 012274 5,21/01/2020,MÉTODO E SERVIDOR DE REDE PARA AUTENTICAÇÃO E ...,H04W 12/04,https://busca.inpi.gov.br/pePI/servlet/Patente...,MÉTODO E SERVIDOR DE REDE PARA AUTENTICAÇÃO ...,H04W 12/04 ; H04W 12/00 ; H04W 12/06,Network method and server for key authenticati...,"{'Produce', 'Move', 'Break Down', 'Change Phas...",Garantia de segurança e autenticação do dispos...,Autenticar e gerenciar chaves,Dispositivo de terminal em rede de comunicação...,Geração e derivação de chaves criptográficas,"O/a ""Proteger"" é um/uma Aplicação, que no/na G..."
4,BR 11 2021 014438 2,23/01/2020,PRESERVAÇÃO DE CÉLULAS-TRONCO,A01N 1/02,https://busca.inpi.gov.br/pePI/servlet/Patente...,PRESERVAÇÃO DE CÉLULAS-TRONCO. A invenção re...,A01N 1/02 ; C12N 5/07,Preservation of Tronco Cells,"{'Produce', 'Move', 'Break Down', 'Preserves',...",Efeito,Preservar,Sólido,Hydration,NaN


In [6]:
import numpy as np

# Filtra registros onde derived_from não é NaN ou vazio
mask_derived = df_patents_sample["derived_from"].notna() & (df_patents_sample["derived_from"] != "")
derived_rows = df_patents_sample[mask_derived]

# Para cada registro com derived_from preenchido
for idx, row in derived_rows.iterrows():
    kind_effect = row["kind_effect"]
    task = row["task"]
    obj = row["object"]
    physical_effect = row["physical_effect"]
    derived_value = row["derived_from"]

    # Busca outros registros com os mesmos campos, mas derived_from vazio ou NaN
    mask_same = (
        (df_patents_sample["kind_effect"] == kind_effect) &
        (df_patents_sample["task"] == task) &
        (df_patents_sample["object"] == obj) &
        (df_patents_sample["physical_effect"] == physical_effect) &
        (df_patents_sample["derived_from"].isna() | (df_patents_sample["derived_from"] == ""))
    )

    # Atualiza derived_from nesses registros
    df_patents_sample.loc[mask_same, "derived_from"] = derived_value

In [7]:
df_patents_sample.to_csv("data/processed/patents_inpi_llm_matched_10percent.csv", index=False)